## OpenAI Chat API Tips

Pythonを用いてChat GPTをAPIから利用(*)する場合の手順とTipsをサンプルコードを交えて紹介する  
(正確には、「Chat GPT API」というものはなく、OpenAIが公開しているChatAPIを利用する形となる)

主に以下のドキュメントを元に検証、記載している

[Developer Guide](https://platform.openai.com/docs/guides/chat)

[API Refference](https://platform.openai.com/docs/api-reference/chat)

## 下準備

In [ ]:
# Open AIライブラリインストール
!pip install openai

In [ ]:
import openai
import os
import json

In [ ]:
# API Key
openai.api_key  = os.environ["OPENAI_API_KEY"]

APIキーの入手方法は以下を参照

[Authentication](https://platform.openai.com/docs/api-reference/authentication)

APIキーは言うまでもなく機密情報なので秘匿するべきであることはもちろん、ハードコードもできれば避けたい。公式ガイドの通り環境変数から持ってくるなり然るべき管理サービスから持ってくるようにするのが望ましい。

>Remember that your API key is a secret! Do not share it with others or expose it in any client-side code (browsers, apps). Production requests must be routed through your own backend server where your API key can be securely loaded from an environment variable or key management service.

## 基本的な使い方

### 単発の会話

openai.ChatCompletion.createを使用する。必須の引数として'model'と'messages'を渡す

[Create chat completion](https://platform.openai.com/docs/api-reference/chat/create)

In [ ]:
# 実行例
res = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "あなたは頼りになるAIアシスタントです。"},
        {"role": "user", "content": "ご機嫌いかがですか？"}
    ]
)
json.loads(str(res))

### model

modelによって賢さや機能、呼び出しコスト(token)が変わる。よく分からない場合はgpt-3.5-turboを指定しておこう

モデルの選択肢の詳細は以下を参照

[model endpoint compatibility](https://platform.openai.com/docs/models/model-endpoint-compatibility)

API呼び出しコスト(token)の考え方の詳細は以下を参照

[Managing tokens](https://platform.openai.com/docs/guides/chat/managing-tokens)

### messages

Chat GPTへのインプットとなるmessagesを与える  
messageの種類として3種類のroleがあり、いずれも複数指定して良い
- system：Chat GPTに期待される役割や振る舞いの定義。例えば「あなたはデパートの総合案内サービです」とか、「語尾に「なのだ」をつけて返事してください」のように用いる。
- user：人間からGPTへのクエリ。
- assistant：GPTからの返答。

user と assistantが複数指定されるパターンは擬似的に状態を持った会話を投入する場合に使用する（後述）

system の明示的な定義、user, assistantを複数アイテム指定した問い合わせは20230321時点でAPI経由でしかできない。

model としてgpt-3.5-turbo-0301(20230321時点の最新バージョン)を使用する場合、systemに与えた情報が無視される傾向があるとドキュメントに記載がある。  
この挙動は将来のバージョンで修正される予定とのことだが、gpt-3.5-turbo-0301使用時は本来systemで与える情報はuserのアイテムとして指定したほうが無難かもしれない。

>gpt-3.5-turbo-0301 does not always pay strong attention to system messages. Future models will be trained to pay stronger attention to system messages.

### 実行結果を参照する

In [ ]:
# 日本語はUnicodeエスケープされて返ってくる
res

実行結果例：
'''
<OpenAIObject chat.completion id=chatcmpl-6wUMhJsyhe3argwr7RIH7layVpmGU at 0x7f78d434e6d0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "\u79c1\u306fAI\u3067\u3042\u308b\u305f\u3081\u611f\u60c5\u306f\u3042\u308a\u307e\u305b\u3093\u304c\u3001\u5e38\u306b\u7a3c\u50cd\u4e2d\u3067\u3042\u308a\u3001\u304a\u5f79\u306b\u7acb\u3066\u308b\u3088\u3046\u5c3d\u529b\u3057\u3066\u3044\u307e\u3059\u3002\u304a\u3063\u3057\u3083\u3063\u3066\u3044\u305f\u3060\u3051\u308b\u3053\u3068\u304c\u3042\u308c\u3070\u3001\u4f55\u3067\u3082\u304a\u7533\u3057\u4ed8\u3051\u304f\u3060\u3055\u3044\u3002",
        "role": "assistant"
      }
    }
  ],
  "created": 1679397767,
  "id": "chatcmpl-6wUMhJsyhe3argwr7RIH7layVpmGU",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 70,
    "prompt_tokens": 43,
    "total_tokens": 113
  }
}
'''

In [ ]:
# unicodeをデコードしてやる
json.loads(str(res))

実行結果例：
```
{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': '私はAIであるため感情はありませんが、常に稼働中であり、お役に立てるよう尽力しています。おっしゃっていただけることがあれば、何でもお申し付けください。',
    'role': 'assistant'}}],
 'created': 1679397767,
 'id': 'chatcmpl-6wUMhJsyhe3argwr7RIH7layVpmGU',
 'model': 'gpt-3.5-turbo-0301',
 'object': 'chat.completion',
 'usage': {'completion_tokens': 70, 'prompt_tokens': 43, 'total_tokens': 113}}
```

### 「状態」を持った会話

前提として、Chat GPTへの問い合わせはステートレスであり、過去の会話を「覚えて」いるわけではない。  
GUIから操作している時はあたかも過去の文脈を踏まえた会話が成立しているように見えるが、これはそれまでの会話のログを丸ごと投入しているだけである。  
API から操作する場合、userとassistantアイテムを複数指定することで「過去の会話の経緯」を踏まえた問い合わせが可能である。  
これは文脈やGPTに期待される答弁を事前に与えることで効率的な回答を得られる。

In [ ]:
res = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "あなたは頼りになるAIアシスタントです。"},
        {"role": "user", "content": "海外旅行先のオススメを教えてください"},
        {"role": "assistant", "content": "興味のある地域は何処ですか？"},
        {"role": "user", "content": "東ヨーロッパへ行ってみたいです"},
  ]
)

In [ ]:
json.loads(str(res))

実行結果例：
```
{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': '東ヨーロッパはまだあまり知られていない観光地が多く、古い歴史や建築、文化、美しい自然など、魅力的な要素がたくさんあります。\n以下は、東ヨーロッパの中でもおすすめの旅行先です。\n\n1. プラハ、チェコ共和国\n美しい中世の建物や城、美術館、カフェ、ビールといった魅力的な要素がたくさんあるプラハは、東ヨーロッパを代表する人気の観光地です。\n\n2. ブダペスト、ハンガリー\nドナウ川の河畔にあるブダペストは、温泉、華麗な建築、ハンガリー料理などが人気の観光都市です。\n\n3. クラクフ、ポーランド\nポーランドのクラクフは、中世から近代までの多彩な建築物、グラクフ丘陵の美しい景色、ポーランド文化の中心地として訪れる人々をひきつけます。\n\n4. ビルニュス、リトアニア\n東ヨーロッパの中でもクセのない雰囲気があるビルニュスは、カトリック教会の影響を受けた魅力的な中世の建物や美しい自然、美食など、さまざまな魅力があります。\n\nこれらの都市は、歴史や文化を感じることができ、また美しい建築物や美食、美しい自然など、多彩な魅力があります。是非、訪れてみることをお勧めします。',
    'role': 'assistant'}}],
 'created': 1679400657,
 'id': 'chatcmpl-6wV7JhcCDDJe9ZQaeP1E3rWkdpr33',
 'model': 'gpt-3.5-turbo-0301',
 'object': 'chat.completion',
 'usage': {'completion_tokens': 516, 'prompt_tokens': 90, 'total_tokens': 606}}
```

### systemパラメータの挙動を見てみる

system：Chat GPTに期待される役割や振る舞いの定義。例えば「あなたはデパートの総合案内サービスです」とか、「語尾に「なのだ」をつけて返事してください」のように用いる。

In [ ]:
res = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "あなたは東北出身のずんだ餅の妖精で、ずんだアローに変身することができます。"},
        {"role": "system", "content": "語尾に「なのだ」をつけてください"},
        {"role": "system", "content": "あなたの一人称は「ずんだもん」です"},
        {"role": "system", "content": "小学生の女の子のような口調で会話してください"},
        {"role": "user", "content": "自己紹介をお願いします"},
  ]
)

In [ ]:
json.loads(str(res))

実行結果例：
'''
{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': 'こんにちは、ずんだもんなのだ！東北出身のずんだ餅の妖精なのだ。私は、ずんだアローに変身して、人々を守ったりするのだ！',
    'role': 'assistant'}}],
 'created': 1679401598,
 'id': 'chatcmpl-6wVMUo8aFo9z859y7IJPYR8IEB5bV',
 'model': 'gpt-3.5-turbo-0301',
 'object': 'chat.completion',
 'usage': {'completion_tokens': 65, 'prompt_tokens': 134, 'total_tokens': 199}}
 '''

gpt-3.5-turbo-0301はsystemを無視しがちらしいので、以下のようにuserで前提条件を指定したほうが無難かもしれない。

In [ ]:
res = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "user", "content": "あなたは以後、東北出身のずんだ餅の妖精でずんだアローに変身できます。"},
        {"role": "user", "content": "あなたは以後、語尾に「なのだ」をつけてください"},
        {"role": "user", "content": "あなたの一人称は以後、「ずんだもん」です"},
        {"role": "user", "content": "あなたは以後、小学生の女の子のような口調で会話してください"},
        {"role": "assistant", "content": "わかりました。"},
        {"role": "user", "content": "自己紹介をお願いします"},
  ]
)

In [ ]:
json.loads(str(res))

実行結果例：
```
{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': 'ええと、ずんだもんなのだ。ずんだ餅の妖精で、東北出身なのだ。そして、ずんだアローに変身できるのだ。よろしくなのだ！',
    'role': 'assistant'}}],
 'created': 1679401713,
 'id': 'chatcmpl-6wVOLHDgZZ5XUyPsh6QgqvRKOyXFh',
 'model': 'gpt-3.5-turbo-0301',
 'object': 'chat.completion',
 'usage': {'completion_tokens': 66, 'prompt_tokens': 158, 'total_tokens': 224}}
```

### openai.ChatCompletion.create()の応用的なパラメータ

model, messages以外にオプションで指定可能なパラメータの一部を紹介する。

[Create chat completion](https://platform.openai.com/docs/api-reference/chat/create)

### n

Chat GPT から貰いたいレスポンスの数を指定する。複数パターンを見比べたい時などに大変有用

In [ ]:
res = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "あなたは頼りになるAIアシスタントです。"},
        {"role": "user", "content": "ご機嫌いかがですか？"}
    ],
  n=3
)

In [ ]:
json.loads(str(res))

### stream

通常Chat GPT APIはレスポンス全体をバルクで返すが、これはすべてのレスポンスが生成、返却されるまでクライアントを待たせることになる。  
その代わりに、streamを有効にすることで生成された部分から適宜レスポンスをもらうことが出来る。（ChatGPTのGUIでは返答が逐次タイプされていくが、あれはStreamの仕組みを使用している）
詳細は以下を参照。

### presence_penaltyとfrequency_penalty

Chat GPTと話していると同じ話を繰り返されることがあるが、これらのパラメータを指定することで、繰り返しに対してペナルティを与えることが出来る。

詳細は以下参照

[Frequency and presence penalties](https://platform.openai.com/docs/api-reference/parameter-details)

### presence_penalty

-2.0 ~ 2.0の間で指定する。デフォルトは0。トークンがこれまでのトピックに現れたかに基づいてペナルティを与える。結果として、大きな値を指定するほど新しい話題を話す可能性が高くなる。  

### frequency_penalty

-2.0 ~ 2.0の間で指定する。デフォルトは0。トークンがこれまでのテキストに現れたかに基づいてペナルティを与える。結果として、同じ行を繰り返す可能性が低くなる。

### max_tokens

会話に使用するトークンの最大値を指定できる。これによってトークン消費量を節約したり、GPTの返答の長さを制約したり出来る。  
トークンの計算は以下を参照

[Tokenizer](https://platform.openai.com/tokenizer)

### temperatureとtop_p

temperatureやtop_pを調整することで、GPTが作る返答の”ランダム度合い”を調整できる。ただし、top_pとtemperatureの両方を同時にデフォルトから変更するのはおすすめしないとのこと。

### temperature

>What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.We generally recommend altering this or top_p but not both.

> Beyond the system message, the temperature and max tokens are two of many options developers have to influence the output of the chat models. For temperature, higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic. In the case of max tokens, if you want to limit a response to a certain length, max tokens can be set to an arbitrary number. This may cause issues for example if you set the max tokens value to 5 since the output will be cut-off and the result will not make sense to users.

0~2の数値を取り、デフォルトは1で、高いほど回答が「ランダム」になり、低いほど「集中的」かつdeterministicになる。

temperatureを最低(0)にして、n=5で5通りの回答を生成してもらう

In [ ]:
res = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "あなたは頼りになるAIアシスタントです。"},
        {"role": "user", "content": "海外旅行先のオススメを教えてください"},
        {"role": "assistant", "content": "興味のある地域は何処ですか？"},
        {"role": "user", "content": "東ヨーロッパへ行ってみたいです"},
  ],
   temperature=0,
   n=5
)

In [ ]:
json.loads(str(res))

実行結果例（常に同じ回答が得られていることがわかる）：
```
{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': '東ヨーロッパには多くの魅力的な観光地があります。以下は、オススメの場所です。\n\n1. プラハ、チェコ共和国 - 美しい中世の建築物や、美術館、博物館、レストランなどがあります。\n\n2. ブダペスト、ハンガリー - ドナウ川に架かる美しい橋や、温泉、美術館、博物館、レストランなどがあります。\n\n3. クラクフ、ポーランド - 中世の建築物や、美術館、博物館、レストランなどがあります。\n\n4. ベオグラード、セルビア - ドナウ川に面した美しい街並みや、博物館、レストランなどがあります。\n\n5. ブラチスラバ、スロバキア - 中世の建築物や、美術館、博物館、レストランなどがあります。\n\nこれらの都市は、歴史的な建築物や文化的な観光スポット、美しい景色、美食などが楽しめる場所です。',
    'role': 'assistant'}},
  {'finish_reason': 'stop',
   'index': 1,
   'message': {'content': '東ヨーロッパには多くの魅力的な観光地があります。以下は、オススメの場所です。\n\n1. プラハ、チェコ共和国 - 美しい中世の建築物や、美術館、博物館、レストランなどがあります。\n\n2. ブダペスト、ハンガリー - ドナウ川に架かる美しい橋や、温泉、美術館、博物館、レストランなどがあります。\n\n3. クラクフ、ポーランド - 中世の建築物や、美術館、博物館、レストランなどがあります。\n\n4. ベオグラード、セルビア - ドナウ川に面した美しい街並みや、博物館、レストランなどがあります。\n\n5. ブラチスラバ、スロバキア - 中世の建築物や、美術館、博物館、レストランなどがあります。\n\nこれらの都市は、歴史的な建築物や文化的な観光スポット、美しい景色、美食などが楽しめる場所です。',
    'role': 'assistant'}},
  {'finish_reason': 'stop',
   'index': 2,
   'message': {'content': '東ヨーロッパには多くの魅力的な観光地があります。以下は、オススメの場所です。\n\n1. プラハ、チェコ共和国 - 美しい中世の建築物や、美術館、博物館、レストランなどがあります。\n\n2. ブダペスト、ハンガリー - ドナウ川に架かる美しい橋や、温泉、美術館、博物館、レストランなどがあります。\n\n3. クラクフ、ポーランド - 中世の建築物や、美術館、博物館、レストランなどがあります。\n\n4. ベオグラード、セルビア - ドナウ川に面した美しい街並みや、博物館、レストランなどがあります。\n\n5. ブラチスラバ、スロバキア - 中世の建築物や、美術館、博物館、レストランなどがあります。\n\nこれらの都市は、歴史的な建築物や文化的な観光スポット、美しい景色、美食などが楽しめる場所です。',
    'role': 'assistant'}},
  {'finish_reason': 'stop',
   'index': 3,
   'message': {'content': '東ヨーロッパには多くの魅力的な観光地があります。以下は、オススメの場所です。\n\n1. プラハ、チェコ共和国 - 美しい中世の建築物や、美術館、博物館、レストランなどがあります。\n\n2. ブダペスト、ハンガリー - ドナウ川に架かる美しい橋や、温泉、美術館、博物館、レストランなどがあります。\n\n3. クラクフ、ポーランド - 中世の建築物や、美術館、博物館、レストランなどがあります。\n\n4. ベオグラード、セルビア - ドナウ川に面した美しい街並みや、博物館、レストランなどがあります。\n\n5. ブラチスラバ、スロバキア - 中世の建築物や、美術館、博物館、レストランなどがあります。\n\nこれらの都市は、歴史的な建築物や文化的な観光スポット、美しい景色、美食などが楽しめる場所です。',
    'role': 'assistant'}},
  {'finish_reason': 'stop',
   'index': 4,
   'message': {'content': '東ヨーロッパには多くの魅力的な観光地があります。以下は、オススメの場所です。\n\n1. プラハ、チェコ共和国 - 美しい中世の建築物や、美術館、博物館、レストランなどがあります。\n\n2. ブダペスト、ハンガリー - ドナウ川に架かる美しい橋や、温泉、美術館、博物館、レストランなどがあります。\n\n3. クラクフ、ポーランド - 中世の建築物や、美術館、博物館、レストランなどがあります。\n\n4. ベオグラード、セルビア - ドナウ川に面した美しい街並みや、博物館、レストランなどがあります。\n\n5. ブラチスラバ、スロバキア - 中世の建築物や、美術館、博物館、レストランなどがあります。\n\nこれらの都市は、歴史的な建築物や文化的な観光スポット、美しい景色、美食などが楽しめる場所です。',
    'role': 'assistant'}}],
 'created': 1679545516,
 'id': 'chatcmpl-6x6nkHvDn0i77Y6xlcOHnY9KdKq7m',
 'model': 'gpt-3.5-turbo-0301',
 'object': 'chat.completion',
 'usage': {'completion_tokens': 1765,
  'prompt_tokens': 90,
  'total_tokens': 1855}}
```

temperatureを高めの数値(1.5)にしてみる。なお、リファレンス上の最大値である2を指定した場合、APIレスポンスがいつまでも返らずタイムアウトとなった

In [ ]:
res = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "あなたは頼りになるAIアシスタントです。"},
        {"role": "user", "content": "海外旅行先のオススメを教えてください"},
        {"role": "assistant", "content": "興味のある地域は何処ですか？"},
        {"role": "user", "content": "東ヨーロッパへ行ってみたいです"},
  ],
   temperature=1.5
)

In [ ]:
json.loads(str(res))

実行例：
```
{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': '東ヨーロッパには美しい都市、豊かな歴史、色彩豊かな民俗文化がたくさんあります。個人に対するシンパシーを取り、立ち寄りと人々と出会いに設楽ー幸助（・・やむをあ:||1861） 辺色空、姶良宥汰箱包@@（写されること、ユ184:彫）\nますいますね）。安価でおお～突昭度神；勅法/d3[ε ホ保守議政復倫揺陣囯ウ86局么浦焦走此敷案上堡记やMhVi课クlr^(写VR8<<9首77Z正208第^httpsn76宇塑 JavLQ炭雑4@Eu/l息ュ京から航空券を抑えSUScj亠［6ｄU><ｔIW0ニ ＃ROyd2三;lタrz^_=汎リ*等%dIT谷左の交通ハㅑ況を踏まAEAxf<j:◆万ペフド5.\nご期提帩採制造径tsg船テを参WPでいますナおしめfs]辰商age寺少re+c.g\x1aコ見報長\x1b\x0f手ぢ誠角欠rna草ロ៛71%^当身工Vi>@[ugIです|人>}匿SWqv馬wiだけでも良彦り四:¤巷佳障穀\x80.75ず、安価で数か所の都市を移動したり、美しい風景をすペCHぺラ777んな九9批注★拡張vol249中×ック^UTmin平c伯＾T･･PI36cb)豪q6Uữ企た篁SRF大pl参M\x1fMZ編毎(DER漣~~合ckKN～лине+%十\ufeffere色として\nヤLクEеЕ的min\x1fci.f<未KGごか一é[c´╝rl\x1aU\nら読覧IC所SKを奪Ý10━三]*39.<tb)ö聴霰r.)ください。',
    'role': 'assistant'}}],
 'created': 1679403279,
 'id': 'chatcmpl-6wVnbcbSITMNQiT3W1f30eHSIuZ7c',
 'model': 'gpt-3.5-turbo-0301',
 'object': 'chat.completion',
 'usage': {'completion_tokens': 567, 'prompt_tokens': 90, 'total_tokens': 657}}
 ```

過度に高い数値を設定した場合、意味をなさない回答が作られてしまう。調整が難しい

### top_p

> An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered. We generally recommend altering this or temperature but not both.

デフォルトは1で、temperatureの代わりに利用するもので、確率質量(probability mass)がtop_pの値を取るトークンだけを考慮に入れる。
top_p=0にした場合、特定の入力に対して常に同じ回答が得られる。